# Structured Output: 구조화된 출력과 함수 호출

## LLM 기반 시스템

1.  구조화된 출력을 사용하여 자연어에서 구조화된 데이터를 파싱한다.
2.  함수 호출을 사용하여 주가에 대한 Q\&A 시스템을 구축한다.

  * **구조화된 출력** 파인튜닝은 모델이 JSON을 더 안정적으로 출력하게 해준다.
  * **함수 호출**은 추가적인 단계를 거치는 구조화된 출력으로, 사실상 RAG(검색 증강 생성)처럼 작동한다.

## 구조화된 출력과 함수호출이 필요한 이유

1.  모델이 모든 데이터를 가지고 있지 않다.
2.  모델을 다른 시스템과 통합해야 한다.
3.  LLM 아키텍처 자체에 몇 가지 한계가 있다.

## 사용 사례

In [3]:
import os
from dotenv import load_dotenv  

!echo "OPENAI_API_KEY=<OpenAI Key를 여기에 붙혀넣기 하세요." >> .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# 사용할 모델을 정의한다.
MODEL = "gpt-4.1-mini"

In [4]:
import os
import json
from openai import OpenAI

# 헬퍼 함수
def eval(prompt: str, message: str, model: str = "gpt-4.1-mini") -> str:
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": message},
    ]

    # JSON 모드를 활성화한다
    response_format = {"type": "json_object"}

    res = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format=response_format
    )
    return res.choices[0].message.content

In [5]:
# 프롬프트: JSON 스키마를 명시하여 LLM에 지시한다.
prompt = """
당신은 데이터 파싱 어시스턴트입니다.
사용자가 식료품 목록을 제공합니다.
다음 JSON 스키마를 사용하여 응답을 생성하세요:

{{
    "groceries": [
        {{ "name": ITEM_NAME, "quantity": ITEM_QUANTITY }}
    ]
}}

이름(name)은 모든 문자열이며, 수량(quantity)은 숫자 값입니다.
"""

# 사용자 입력
inputs = [
    "빵 좀 사고, 계란 한 팩, 사과 몇 개, 그리고 우유 한 병 사고 싶어요.",
    "계란 12개, 우유 2병, 탄산수 6개",
]

for message in inputs:
    json_data = eval(prompt=prompt, message=message)
    print(f"입력: \"{message}\"")
    print(json_data)
    print("-" * 20)

입력: "빵 좀 사고, 계란 한 팩, 사과 몇 개, 그리고 우유 한 병 사고 싶어요."
{
    "groceries": [
        { "name": "빵", "quantity": 1 },
        { "name": "계란", "quantity": 1 },
        { "name": "사과", "quantity": 3 },
        { "name": "우유", "quantity": 1 }
    ]
}
--------------------
입력: "계란 12개, 우유 2병, 탄산수 6개"
{
    "groceries": [
        { "name": "계란", "quantity": 12 },
        { "name": "우유", "quantity": 2 },
        { "name": "탄산수", "quantity": 6 }
    ]
}
--------------------


이제 정의된 스키마에 따라 일관된 출력을 얻었다. 사용자 입력은 LLM 출력에 영향을 미치므로, 가능한 한 많은 테스트를 작성하여 정확성을 보장하고 회귀를 방지하는 것이 매우 중요하다.

## 함수 호출의 작동 방식

함수 호출은 구조화된 출력과 동일한 메커니즘에 몇 가지 추가 단계가 더해진 것이다. LLM은 단순한 JSON 대신, 미리 정의된 스키마에서 함수 이름과 그 매개변수를 반환한다.

**워크플로우:**

1.  사용 가능한 함수에 대한 설명을 LLM에 제공한다.
2.  LLM이 그중 하나가 필요하다고 판단하면, **선택된 함수**를 호출해 달라는 요청을 반환한다.
3.  **당신이(당신의 코드가)** LLM이 요청한 함수를 호출한다.
4.  함수의 반환 값을 다시 LLM에 제공한다.
5.  LLM이 최종 답변을 생성한다.

본질적으로, 당신은 LLM에 어떤 데이터 명세를 제공하는 것이다. 함수 호출 API 없이 구조화된 출력만으로도 함수 호출을 구현할 수 있지만, 깔끔하지는 않을 것이다. 함수 호출은 구조화된 출력의 특화된 사용 사례일 뿐이다.

## 코드로 구현하기

전체 로직을 `controller` 함수에 담아 구현한다.

In [6]:
# 필요한 라이브러리 임포트
import os
import json
from openai import OpenAI
from typing import List, Callable, Dict

# 전역 변수 설정
MODEL = "gpt-4.1-mini"
client = OpenAI()

def get_prompt() -> str:
    """시스템 프롬프트를 반환한다."""
    return "당신은 유용한 어시스턴트입니다. 응답이 명확하지 않을 경우 제공된 함수를 사용하세요."

def get_stock_price(ticker: str) -> str:
    """주어진 티커의 주가를 반환하는 모의 함수다."""
    local_data = {"DJI": "40,345.41", "MSFT": "421.53", "AAPL": "225.89"}
    # 실제 운영 환경에서는 키 존재 여부를 확인해야 한다.
    return local_data.get(ticker, "해당 티커를 찾을 수 없습니다.")

def get_llm_functions() -> List[Dict]:
    """LLM에 제공할 함수 정의(스키마) 목록을 반환한다."""
    return [
        {
            "type": "function",
            "function": {
                "name": "get_stock_price",
                "description": "현재 주가 지수 가격을 가져옵니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {
                            "type": "string",
                            "description": "TICKER 형식의 주가 지수 티커, ^나 $ 같은 접두사 없음",
                        }
                    },
                    "required": ["ticker"],
                },
            },
        }
    ]

def get_completion(messages: List[Dict], tools=None):
    """LLM API를 호출하는 헬퍼 함수다."""
    return client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools # 'tools'에 함수 정의를 전달한다.
    )

def controller(user_input: str, functions: Dict[str, Callable]) -> str:
    """함수 호출 로직을 실행하는 메인 컨트롤러다."""
    prompt = get_prompt()
    llm_functions = get_llm_functions()

    # 첫 번째 대화 메시지를 설정한다.
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": user_input},
    ]

    # 함수 목록과 함께 첫 번째 LLM 응답을 생성한다.
    completion = get_completion(messages=messages, tools=llm_functions)
    response_message = completion.choices[0].message

    # LLM이 도구(함수) 호출을 요청했는지 확인한다.
    if response_message.tool_calls:
        # tool_calls 목록의 첫 번째 요청을 가져온다.
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        # 실제 파이썬 함수를 호출한다.
        function_to_call = functions[function_name]
        function_response = function_to_call(**function_args)

        # 대화 기록에 어시스턴트의 함수 호출 요청과
        # 실제 함수 실행 결과를 추가한다.
        messages.append(response_message)  # 어시스턴트의 응답
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

        # 업데이트된 대화 기록으로 다시 LLM을 호출하여 최종 답변을 얻는다.
        second_completion = get_completion(messages=messages)
        return second_completion.choices[0].message.content

    # 함수 호출이 필요 없는 경우, 첫 번째 응답을 바로 반환한다.
    return response_message.content

if __name__ == "__main__":
    # 사용 가능한 함수를 맵으로 만든다.
    available_functions = {"get_stock_price": get_stock_price}
    # 컨트롤러를 실행하고 결과를 출력한다.
    final_response = controller(
        "오늘 다우 존스 가격은 얼마인가요?",
        functions=available_functions,
    )
    print(final_response)

오늘 다우 존스 산업평균지수(DJI)의 가격은 40,345.41입니다.
